# PyKX Query Components
Query all components, RDB, HDB, and Gateway.


In [24]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

from env import *
import pykx as kx

from managed_kx import *

from basictick_setup import *

In [25]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    os.system(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# Connections to Clusters

In [26]:
# @todo: use the dictionary of clusters (all_clusters)
# key is the type/role of cluster, name is what you use below for that connection

hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)
tp  = get_pykx_connection(client, 
                           environmentId=ENV_ID, clusterName=TP_CLUSTER_NAME, 
                           userName=KDB_USERNAME, boto_session=session)
rdb = get_pykx_connection(client, 
                           environmentId=ENV_ID, clusterName=RDB_CLUSTER_NAME, 
                           userName=KDB_USERNAME, boto_session=session)
calc = get_pykx_connection(client, 
                           environmentId=ENV_ID, clusterName=CALC_CLUSTER_NAME, 
                           userName=KDB_USERNAME, boto_session=session)
gw = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=GW_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

:tcps://vpce-0866c4591a39a311b-88a0x71t.vpce-svc-030aabbf355f2be37.us-east-1.vpce.amazonaws.com:443:cgervin:Host=vpce-0866c4591a39a311b-88a0x71t.vpce-svc-030aabbf355f2be37.us-east-1.vpce.amazonaws.com&Port=443&User=cgervin&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjENb%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJGMEQCIGtM%2B%2FbeAdTYQC9%2FE2P0Lij71PRTyWFmPiV0BrqbYdaAAiBtDe3dYgA3Ey11LT8bvDa34HSXM78v4G8aCLA6KA2ptCr4AggvEAEaDDU5MDc4MDYxNTI2NCIMCntodSNv80eJxPatKtUCj6%2FxLcBaPl7BLAJAjcYsV6xigfb0jxAj%2FM%2F%2FK039i5G9u8WFGGh%2BM4mNOnpyAwWiCG9YD8DR6SCd%2FxiU%2BPGebtunSiEU%2FanJ60puDmSqTIE1p9Kov9dJbCP1M7U6YJB7lYbqMf1N65YD2yRlPL520Q9OLDu5B074Qhd2Hgwq0%2Fbnm2%2FoI9v%2B0%2BBOc5lOu%2BctJeMOUt78A%2FiXYbtTXdWYqOnFSaUOFaXPpLjEl0x%2F2B25i0hpLB0GzGb2c7Sem%2FilDo%2BYKhp5fpxlIeuVYaTthNmQkYCGP5YHG4kBfkzMzNHtnPGyFtlwT6NjVvIpTqnBGm06P%2FqRtfJPA2S1xdQ7eF0IjPRz4gRRgdA9BMWKKgonSm8yU8OUeng%2F0N7mOm5Z5TVux1tpAuSdOfIIwXqe9hcsEDcTzOdbXyRD21s%2BDgSYsrk09nS1ert%2BEPWhIkJHUMa5fL4w9OrDs

# Current State of HDB

In [9]:
# Dates and Counts
hdb_pdf = hdb("select counts:count i by date from trade").pd()

hdb_head_pdf = hdb("select from trade where date = min date, i<5").pd()

# Number of Rows
hdb_rows = hdb("count select from trade where date=max date").py()

display(hdb_pdf)

hdb_rows

,counts
date,
2024-04-19,866361
2024-04-22,888436
2024-04-23,888187
2024-04-24,883938
2024-04-25,889931
2024-04-26,884716


884716

# Current State of RDB

In [29]:
# Query the RDB
rdb_rows_all = rdb("`trade`quote!count each value each `trade`quote").py()

rdb_head_pdf = rdb("select [5] from trade").pd()

rdb_tail_pdf = rdb("select [-5] from trade").pd()

# Number of Rows
rdb_rows = rdb_rows_all['trade']

rdb_rows_all

{'trade': 13169, 'quote': 65553}

In [9]:
rdb("trade")

,time,sym,price,size
,,,,
0,2024.04.29D14:27:37.131725750,AMD,33.7,77
1,2024.04.29D14:27:37.131725750,DOW,20.02,93
2,2024.04.29D14:27:37.131725750,ORCL,35.99,48
3,2024.04.29D14:27:37.131725750,SBUX,63.06,12
4,2024.04.29D14:27:37.131725750,GOOG,72.6,70
5,2024.04.29D14:27:37.131725750,HPQ,36f,33
6,2024.04.29D14:27:37.131725750,GOOG,72.57,10
7,2024.04.29D14:27:37.131725750,GOOG,72.57,80
8,2024.04.29D14:27:37.131725750,SBUX,63.09,84


In [12]:
#rdb('trade:0#trade;quote:0#quote')
rdb("""
    generateTWAP:{[syms;st;et] 
        if[syms~`;syms:exec distinct sym from trade];
        // Calculate statistics from trade and quote tables, join the tables with 
        // appropriate join function in this case a union join 
        quoteMetrics:select avg_spread:avg (ask-bid),twa_spread:(next[time]- time) wavg (ask-bid), avg_size:0.5*avg (asize+bsize),avg_duration:"t"$avg next[time]-time by sym from quote where sym in syms,time within(st;et); 
        tradeMetrics:select std_dev:2*dev price, twap:(next[time]-time) wavg price,max_price:max price, min_price:min price,vwap:size wavg price by sym from trade where sym in syms,time within(st;et); 
        quoteMetrics uj tradeMetrics 
     }
""")

rdb("generateTWAP[`;08:00:00.040; 19:00:00.000]")
rdb("generateTWAP[`AAPL`IBM;08:00:00.040; 19:00:00.000]")

,avg_spread,twa_spread,avg_size,avg_duration,std_dev,twap,max_price,min_price,vwap
sym,,,,,,,,,
AAPL,1.000138,0.8400066,54.42035,04:34:52.033,2.301324,83.34383,86.17,81.25,83.57222
IBM,0.9873642,1.020652,54.14728,11:32:28.961,0.7684107,42.51837,43.3,41.79,42.42548


# Current State of HDB

In [12]:
hdb('.Q.lo[hsym`$.aws.akdbp,"/",.aws.akdb,"/";0b;0b]')
hdb("select count i by date from quote where date in 10#desc date")

,x
date,
2024.04.19,4323449
2024.04.22,4440838
2024.04.23,4446429
2024.04.24,4422176
2024.04.25,4447795
2024.04.26,4424949
2024.04.29,65553


# Current State of GW

In [34]:
gw('{x"max date"} each {exec handle from .conn.procs where process=`hdb}[]')

pykx.DateVector(pykx.q('2024.04.26 2024.04.29 2024.04.26'))

In [37]:
gw('init')

pykx.Lambda(pykx.q('
{[hdb_name; rdb_name]
    zx: .gw.getDataNodes[hdb_name; rdb_name];

    .gw.connectToDataNodes[zx];

    .awscust.z.ts{[]
        /If not all nodes connected - attempt to reconnect
        if[not all .conn.procs`connected;
            show"Attempting to connect to disconnected data nodes...";
            .conn.connectDisconnected[];
            if[all .conn.procs`connected;
                show"All connected!";
                ];
            ];
        };

    .awscust.z.pc:.conn.handleDrop;

    /Attempt recon every 10s
    system"t 10000";
    }
'))

In [33]:
# Query the GW for its connected processes, expect there to be connected processes as well
proc_pdf = gw("select process, handle, connected, address from .conn.procs").pd()

# are any processes not connected? if so-reconnect
if (len(proc_pdf) == 0) or len(proc_pdf[proc_pdf.connected == False].index) > 0:
    print("reinit Gateway")
    gw("reinit[hdb_name; rdb_name]")
    proc_pdf = gw("select process, handle, connected, address from .conn.procs").pd()
    
# truncate with elipsis
proc_pdf['address'] = proc_pdf['address'].str.slice(0,110)+"..."

display(proc_pdf)

,process,handle,connected,address
0,rdb,13,True,:tcps://ip-192-168-12-30.ec2.internal:443:GATEWAY_basictickdb:Host=ip-192-168-12-30.ec2.internal&Port=443&User...
1,rdb,14,True,:tcps://ip-192-168-6-88.ec2.internal:443:GATEWAY_basictickdb:Host=ip-192-168-6-88.ec2.internal&Port=443&User=G...
2,hdb,18,True,:tcps://ip-192-168-1-11.ec2.internal:443:GATEWAY_basictickdb:Host=ip-192-168-1-11.ec2.internal&Port=443&User=G...
3,hdb,19,True,:tcps://ip-192-168-2-187.ec2.internal:443:GATEWAY_basictickdb:Host=ip-192-168-2-187.ec2.internal&Port=443&User...
4,hdb,20,True,:tcps://ip-192-168-2-98.ec2.internal:443:GATEWAY_basictickdb:Host=ip-192-168-2-98.ec2.internal&Port=443&User=G...


In [17]:
# query GW
gw("res: `time xasc queryData[`trade;`;.z.D-3;.z.D+1]").pd()

#gw_pdf = gw("res").pd()
gw_head_pdf = gw("select [5] from res").pd()
gw_tail_pdf = gw("select [-5] from res").pd()

# Number of Rows
gw_rows = gw("count res").py()

#
display( gw_head_pdf )
display( gw_tail_pdf )
display( gw_rows )

,sym,time,price,size,source
0,SBUX,2024-04-26 09:30:00.000003557,64.83,93,HDB
1,HPQ,2024-04-26 09:30:00.000046186,37.39,71,HDB
2,GOOG,2024-04-26 09:30:00.000050642,76.40,20,HDB
3,PEP,2024-04-26 09:30:00.000078017,23.64,85,HDB
4,HPQ,2024-04-26 09:30:00.000097458,37.40,86,HDB


,sym,time,price,size,source
0,SBUX,2024-04-29 14:29:47.138424190,63.58,97,RDB
1,AAPL,2024-04-29 14:29:47.138424190,84.39,48,RDB
2,DELL,2024-04-29 14:29:47.138424190,12.29,75,RDB
3,AMD,2024-04-29 14:29:47.138424190,33.86,90,RDB
4,GOOG,2024-04-29 14:29:47.138424190,74.33,86,RDB


897885

In [50]:
hdb(".query.data[`trade;`IBM;.z.P-5D;.z.P]")

,time,sym,price,size,source
,,,,,
0,2024.04.24D14:41:54.324671028,IBM,47f,48,HDB
1,2024.04.24D14:41:56.597749293,IBM,46.96,75,HDB
2,2024.04.24D14:41:58.280569262,IBM,47.03,15,HDB
3,2024.04.24D14:41:58.912840859,IBM,46.98,21,HDB
4,2024.04.24D14:41:59.696988164,IBM,47.01,87,HDB
5,2024.04.24D14:42:01.861269323,IBM,47.09,39,HDB
6,2024.04.24D14:42:02.908014805,IBM,47.13,44,HDB
7,2024.04.24D14:42:03.420722874,IBM,47.14,26,HDB
8,2024.04.24D14:42:04.500357123,IBM,47.13,40,HDB


# HDB Contents

In [18]:
display(hdb_pdf)
display(hdb_head_pdf)

# Number of Rows
print(f"Rows: {hdb_rows:,}")

,counts
date,
2024-04-19,866361
2024-04-22,888436
2024-04-23,888187
2024-04-24,883938
2024-04-25,889931
2024-04-26,884716


,date,time,sym,price,size
0,2024-04-19,2024-04-19 09:30:00.001669210,AAPL,84.03,15
1,2024-04-19,2024-04-19 09:30:00.001829581,AAPL,84.07,85
2,2024-04-19,2024-04-19 09:30:00.002491728,AAPL,84.04,95
3,2024-04-19,2024-04-19 09:30:00.003382064,AAPL,83.93,95
4,2024-04-19,2024-04-19 09:30:00.003737960,AAPL,83.93,83


Rows: 884,716


# RDB Contents

In [19]:
display(rdb_head_pdf)
display(rdb_tail_pdf)

# Number of Rows
print(f"Rows: {rdb_rows:,}")

,time,sym,price,size
0,2024-04-29 14:27:37.131725750,AMD,33.70,77
1,2024-04-29 14:27:37.131725750,DOW,20.02,93
2,2024-04-29 14:27:37.131725750,ORCL,35.99,48
3,2024-04-29 14:27:37.131725750,SBUX,63.06,12
4,2024-04-29 14:27:37.131725750,GOOG,72.60,70


,time,sym,price,size
0,2024-04-29 14:29:47.138424190,SBUX,63.58,97
1,2024-04-29 14:29:47.138424190,AAPL,84.39,48
2,2024-04-29 14:29:47.138424190,DELL,12.29,75
3,2024-04-29 14:29:47.138424190,AMD,33.86,90
4,2024-04-29 14:29:47.138424190,GOOG,74.33,86


Rows: 13,169


# GW Contents

In [20]:
display(gw_head_pdf)
display(gw_tail_pdf)

# Number of Rows
print(f"  Gateway Rows: {gw_rows:,}")

# Number of Rows
print(f"HDB + RDB Rows: {hdb_rows+rdb_rows:,}")

# Note: HDB+RDB will have less than GW  b/c data is always arriving to the RDB
# RDB alone was queried first and then again as part of GW query

,sym,time,price,size,source
0,SBUX,2024-04-26 09:30:00.000003557,64.83,93,HDB
1,HPQ,2024-04-26 09:30:00.000046186,37.39,71,HDB
2,GOOG,2024-04-26 09:30:00.000050642,76.40,20,HDB
3,PEP,2024-04-26 09:30:00.000078017,23.64,85,HDB
4,HPQ,2024-04-26 09:30:00.000097458,37.40,86,HDB


,sym,time,price,size,source
0,SBUX,2024-04-29 14:29:47.138424190,63.58,97,RDB
1,AAPL,2024-04-29 14:29:47.138424190,84.39,48,RDB
2,DELL,2024-04-29 14:29:47.138424190,12.29,75,RDB
3,AMD,2024-04-29 14:29:47.138424190,33.86,90,RDB
4,GOOG,2024-04-29 14:29:47.138424190,74.33,86,RDB


  Gateway Rows: 897,885
HDB + RDB Rows: 897,885


# To Manually Update the HDB Cluster to the latest DataView

In [23]:
#wait for dataview status = `ACTIVE.

rdb('updateCluster[.rdb.hdbProc;.rdb.database;.rdb.dbView;"NO_RESTART"]');